In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# This file was created 01_UKB_pull_med_data.ipynb
! dx download data/final_drug_may_2024.csv

In [ ]:
# Load df
df = pd.read_csv('final_drug_may_2024.csv', low_memory = False)
df

In [ ]:
#Create df just of drug names - N=89507

#Drop duplicated drug names
drugs = df.drop_duplicates(subset = 'drug_name', keep = 'first')
drugs = drugs[['drug_name']]

#Make everything lower case
drugs['new_name'] = drugs['drug_name'].str.lower()

#Remove all spaces -- necessary because there are sometimes leading or trailing spaces
drugs['new_name2'] = drugs['new_name'].str.replace(' ', '')

drugs

In [ ]:
# Download our cleaned notebooks -- https://docs.google.com/spreadsheets/d/1b5TvcW0W6dRbv6Gi46hfDPZUwsLnwK8rAv4xrk0c050/edit?usp=sharing
#!dx download data/medications_n1292.csv
!dx download data/omit_keyword_n100.csv
#!dx download data/omit_med_n336.csv

In [ ]:
#Load tables we created
import pandas as pd 
s1 = pd.read_csv('omit_keyword_n100.csv')
s2 = pd.read_csv('omit_med_n336.csv')
s3 = pd.read_csv('medications_n1292.csv')

# First we will omit certain key words

In [ ]:
s = pd.concat([s1])

#make sure cleaned_med and short_name are both lower case
s = s[~s['short_name'].isna()]
s = s[~s['cleaned_med'].isna()]

#Make everything lower case
s['short_name'] = s['short_name'].str.lower()
s['cleaned_med'] = s['cleaned_med'].str.lower()

#Remove spaces
s['short_name'] = s['short_name'].str.replace(' ', '')
s['cleaned_med'] = s['cleaned_med'].str.replace(' ', '')
s

In [ ]:
#Select columns
s = s[['cleaned_med', 'short_name']]
list_drugs = list(s['cleaned_med'])
print(len(list_drugs))
print(list_drugs[0:5])
s

In [ ]:
s.short_name.value_counts()

In [ ]:
# Create a regular expression pattern to match any of the strings in 'strings_to_check'
pattern = '|'.join(list_drugs)

# Extract the matched string into a new column
matches = drugs['new_name2'].str.extract(f'({pattern})', expand=False)
 
# Print the extracted matches
print(matches)

# Assign the extracted matches to a new column
drugs['omit_keyword'] = matches

In [ ]:
# look at df
drugs

In [ ]:
# we want to only keep any drugs that were NOT on the omit_keyword list
drugs = drugs[drugs['omit_keyword'].isna()]
drugs

# Next we will omit certain medications (vitamins, hormones, combo drugs)

In [ ]:
s = pd.concat([s2])

#make sure cleaned_med and short_name are both lower case
s = s[~s['short_name'].isna()]
s = s[~s['cleaned_med'].isna()]

#Make everything lower case
s['short_name'] = s['short_name'].str.lower()
s['cleaned_med'] = s['cleaned_med'].str.lower()

#Remove spaces
s['short_name'] = s['short_name'].str.replace(' ', '')
s['cleaned_med'] = s['cleaned_med'].str.replace(' ', '')
s

In [ ]:
#Select columns
s = s[['cleaned_med', 'short_name']]
list_drugs = list(s['cleaned_med'])
print(len(list_drugs))
print(list_drugs[0:5])
s

In [ ]:
s.short_name.value_counts()

In [ ]:
# Create a regular expression pattern to match any of the strings in 'strings_to_check'
pattern = '|'.join(list_drugs)

# Extract the matched string into a new column
matches = drugs['new_name2'].str.extract(f'({pattern})', expand=False)
 
# Print the extracted matches
print(matches)

# Assign the extracted matches to a new column
drugs['omit_drug'] = matches[0]

In [ ]:
# view df
drugs

In [ ]:
# We only want to keep drugs that were NOT on our omit list
drugs = drugs[drugs['omit_drug'].isna()]
drugs

# Finally we assign a short name to all remaining drugs, allowing us to group drugs with different names by their generic ingredient

In [ ]:
s = pd.concat([s3])

#make sure cleaned_med and short_name are both lower case
s = s[~s['short_name'].isna()]
s = s[~s['cleaned_med'].isna()]

#Make everything lower case
s['short_name'] = s['short_name'].str.lower()
s['cleaned_med'] = s['cleaned_med'].str.lower()

#Remove spaces
s['short_name'] = s['short_name'].str.replace(' ', '')
s['cleaned_med'] = s['cleaned_med'].str.replace(' ', '')
s

In [ ]:
#Select columns
s = s[['cleaned_med', 'short_name']]
list_drugs = list(s['cleaned_med'])
print(len(list_drugs))
print(list_drugs[0:5])
s

In [ ]:
s.short_name.value_counts()

In [ ]:
# Create a regular expression pattern to match any of the strings in 'strings_to_check'
pattern = '|'.join(list_drugs)

# Extract the matched string into a new column
matches = drugs['new_name2'].str.extract(f'({pattern})', expand=False)
 
# Print the extracted matches
print(matches)

# Assign the extracted matches to a new column
drugs['cleaned_med'] = matches

In [ ]:
# View df
drugs

In [ ]:
# For this step, we only want to keep drugs that DO have an assigned cleaned_med name
match = drugs[~drugs['cleaned_med'].isna()]
match

In [ ]:
# Merge this matching table with our table above that contains a short_name for each clenaed_med
drugs = match.merge(s, left_on = 'cleaned_med', right_on = 'cleaned_med', how = 'left')
drugs

In [ ]:
# Create a matching table with the relevant columns
table = drugs[['drug_name', 'new_name', 'cleaned_med', 'short_name']]
table

# Finally, we add these created short_names to the original file

In [ ]:
#Reload the original 56 millow row file
df = pd.read_csv('final_drug_may_2024.csv', low_memory = False)
df

In [ ]:
# Select columns
df = df[['ID', 'issue_date', 'drug_name']]

#Make name lower case
df['new_name'] = df['drug_name'].str.lower()

df

In [ ]:
# Table to match
s = table[['new_name', 'cleaned_med', 'short_name']]
s = s.drop_duplicates(subset = 'new_name')
s

In [ ]:
#Create list of new_names
new_name_list = list(s['new_name'])
print(len(new_name_list))

In [ ]:
# Merge with table 
drugs = df.merge(s, left_on = 'new_name', right_on = 'new_name', how = 'left')
drugs

In [ ]:
#Select drugs that have a short name
drugs_done = drugs[~drugs['short_name'].isna()]
drugs_done

In [ ]:
#Number of unique drugs
test = pd.DataFrame(drugs_done.short_name.value_counts())
test

In [ ]:
drugs_done.to_csv('drugs_with_short_name_OCT_09_24.csv', header = True, index = False)

In [ ]:
! dx upload 'drugs_with_short_name_OCT_09_24.csv' --path data/drugs_with_short_name_OCT_09_24.csv

In [ ]:
#Leftover drugs
left_over = drugs[drugs['short_name'].isna()]
left_over

In [ ]:
left_over.to_csv('left_over.csv', header = True, index = False)

In [ ]:
! dx upload left_over.csv --path data/left_over2.csv

# We did not account for only 328,470 rows out of 56,207,882 rows of data -- which meants we labeled or purposefully omitted over 99% of the available medication data in UKB!!